# Arctic Lake Bathymetry
#### Identify ICESat-13 ATL 13 Granules, convert HDF to CSV - ICESat
#### Melanie Frost
#### 4/18/2023

In [1]:
import datetime
import json
import warnings

import certifi
import urllib3
from urllib.parse import urlencode

from pprint import pprint

import os
import numpy as np
import pandas as pd
from dateutil import parser
import h5py


# Get list of ATL13 granule names

In [2]:
# -----------------------------------------------------------------------------
# class CmrProcess
#
# @author: Caleb Spradlin, caleb.s.spradlin@nasa.gov
# @version: 12.30.2021
#
# https://cmr.earthdata.nasa.gov/search/
# https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html
# -----------------------------------------------------------------------------
class CmrProcess(object):

    CMR_BASE_URL = 'https://cmr.earthdata.nasa.gov' +\
        '/search/granules.umm_json_v1_4?'

    # Range for valid lon/lat
    LATITUDE_RANGE = (-90, 90)
    LONGITUDE_RANGE = (-180, 180)

    # -------------------------------------------------------------------------
    # __init__
    # -------------------------------------------------------------------------
    def __init__(self,
                 mission,
                 dateTime,
                 lonLat=None,
                 error=False,
                 dayNightFlag='',
                 pageSize=150,
                 maxPages=50):

        self._error = error
        self._dateTime = dateTime
        self._mission = mission
        self._pageSize = pageSize
        self._maxPages = maxPages
        
        self._lonLat = lonLat
        self._dayNightFlag = dayNightFlag

    # -------------------------------------------------------------------------
    # run()
    #
    # Given a set of parameters on init (time, location, mission), search for
    # the most relevant file. This uses CMR to search metadata for
    # relevant matches.
    # -------------------------------------------------------------------------
    def run(self):
        print('Starting query')
        outout = set()
        for i in range(self._maxPages):
            
            d, e = self._cmrQuery(pageNum=i+1)
            
            if e and i > 1:
                return sorted(list(outout))
            
            if not e:
                print('Results found on page: {}'.format(i+1))
                out = [r['file_url'] for r in d.values()]
                outout.update(out)
                
        outout = sorted(list(outout))
        return outout
        

    # -------------------------------------------------------------------------
    # cmrQuery()
    #
    # Search the Common Metadata Repository(CMR) for a file that
    # is a temporal and spatial match.
    # -------------------------------------------------------------------------
    def _cmrQuery(self, pageNum=1):

        requestDictionary = self._buildRequest(pageNum=pageNum)
        totalHits, resultDictionary = self._sendRequest(requestDictionary)

        if self._error:
            return None, self._error

        if totalHits <= 0:
            print('No hits on page number: {}, ending search.'.format(pageNum))
            #warnings.warn(msg)
            return None, True

        resultDictionaryProcessed = self._processRequest(resultDictionary)
        return resultDictionaryProcessed, self._error

    # -------------------------------------------------------------------------
    # buildRequest()
    #
    # Build a dictionary based off of parameters given on init.
    # This dictionary will be used to encode the http request to search
    # CMR.
    # -------------------------------------------------------------------------
    def _buildRequest(self, pageNum=1):
        requestDict = dict()
        requestDict['page_num'] = pageNum
        requestDict['page_size'] = self._pageSize
        requestDict['concept_id'] = self._mission
        requestDict['bounding_box'] = self._lonLat
        requestDict['day_night_flag'] = self._dayNightFlag
        requestDict['temporal'] = self._dateTime
        return requestDict

    # -------------------------------------------------------------------------
    # _sendRequest
    #
    # Send an http request to the CMR server.
    # Decode data and count number of hits from request.
    # -------------------------------------------------------------------------
    def _sendRequest(self, requestDictionary):
        with urllib3.PoolManager(cert_reqs='CERT_REQUIRED',
                                 ca_certs=certifi.where()) as httpPoolManager:
            encodedParameters = urlencode(requestDictionary, doseq=True)
            requestUrl = self.CMR_BASE_URL + encodedParameters
            try:
                requestResultPackage = httpPoolManager.request('GET',
                                                               requestUrl)
            except urllib3.exceptions.MaxRetryError:
                self._error = True
                return 0, None

            requestResultData = json.loads(
                requestResultPackage.data.decode('utf-8'))
            status = int(requestResultPackage.status)

            if not status == 400:
                totalHits = len(requestResultData['items'])
                return totalHits, requestResultData

            else:
                msg = 'CMR Query: Client or server error: ' + \
                    'Status: {}, Request URL: {}, Params: {}'.format(
                        str(status), requestUrl, encodedParameters)
                warnings.warn(msg)
                return 0, None

    # -------------------------------------------------------------------------
    # _processRequest
    #
    # For each result in the CMR query, unpackage relevant information to
    # a dictionary. While doing so set flags if data is not desirable (too
    # close to edge of dataset).
    #
    #  REVIEW: Make the hard-coded names class constants? There are a lot...
    # -------------------------------------------------------------------------
    def _processRequest(self, resultDict):

        resultDictProcessed = dict()

        for hit in resultDict['items']:

            fileName = hit['umm']['RelatedUrls'][0]['URL'].split(
                '/')[-1]

            # ---
            # These are hardcoded here because the only time these names will
            # ever change is if we changed which format of metadata we wanted
            # the CMR results back in.
            #
            # These could be placed as class constants in the future.
            # ---
            fileUrl = hit['umm']['RelatedUrls'][0]['URL']
            temporalRange = hit['umm']['TemporalExtent']['RangeDateTime']
            dayNight = hit['umm']['DataGranule']['DayNightFlag']

 
            spatialExtent = hit['umm']['SpatialExten' +
                                          't']['HorizontalSpatialDom' +
                                               'ain']

            key = fileName

            resultDictProcessed[key] = {
                'file_name': fileName,
                'file_url': fileUrl,
                'temporal_range': temporalRange,
                'spatial_extent': spatialExtent,
                'day_night_flag': dayNight}

        return resultDictProcessed

Alaska North Slope

In [3]:
search_dict_AKNS = {
    'site_name': 'Alaska North Slope',
    'bbox': [-168, 67.3, -141, 73], 
    'minmonth': "07",
    'maxmonth': "09",
    'years_list': [2018,2019,2020,2021,2022]
}

In [4]:
# Choose a site
search_dict = search_dict_AKNS

In [5]:
# Find this at https://search.earthdata.nasa.gov/
# search for ATL## in search box, click on granule, click on "view details" 
COLLECTID_ATL13_V5 = "C2153575088-NSIDC_CPRD"

# Page size: 150, number of results returned by page.
PAGESIZE = 150 
# Max page, number of pages to return before ending query.
MAXPAGE = 60
# Total max results will be PAGESIZE * MAXPAG

cmrP_list = []
for YEAR in search_dict['years_list']:
    cmrP = CmrProcess(mission = COLLECTID_ATL13_V5, 
                      dateTime=f"{YEAR}-{search_dict['minmonth']}-01T00:00:00Z,{YEAR}-{search_dict['maxmonth']}-30T23:59:59Z", 
                      lonLat = ','.join(str(e) for e in search_dict['bbox']),
                      pageSize=PAGESIZE,
                      maxPages=MAXPAGE)
    cmrP_list.append(cmrP)

Run Search

In [6]:
resultList = [cmrP.run() for cmrP in cmrP_list]

Starting query
No hits on page number: 1, ending search.
No hits on page number: 2, ending search.
No hits on page number: 3, ending search.
Starting query
Results found on page: 1
Results found on page: 2
No hits on page number: 3, ending search.
Starting query
Results found on page: 1
Results found on page: 2
No hits on page number: 3, ending search.
Starting query
Results found on page: 1
Results found on page: 2
No hits on page number: 3, ending search.
Starting query
Results found on page: 1
Results found on page: 2
No hits on page number: 3, ending search.


In [7]:
len(resultList[0])
atl13_granule_list = [item for sublist in resultList for item in sublist]
print(f"{len(atl13_granule_list)} granules in search results list")

# Get list of just granule names without paths
atl13_granule_list_nopath = [g.split('/')[-1] for g in atl13_granule_list]


782 granules in search results list


Here is a dataframe of the granule paths

In [8]:
atl13_h5_fn_df = pd.DataFrame(atl13_granule_list, columns = ['path'])
atl13_h5_fn_df['site_name'] = search_dict['site_name']

atl13_h5_fn_df

,path,site_name
0,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
1,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
2,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
3,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
4,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
...,...,...
777,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
778,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
779,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope
780,https://data.nsidc.earthdatacloud.nasa.gov/nsi...,Alaska North Slope


In [9]:
granule_test = atl13_granule_list_nopath[0:5]

# Extract HDF files as CSV

In [10]:
print(os.getcwd())
directory = r'/css/icesat-2/ATLAS/ATL13.005'
files = os.listdir(directory)
os.chdir(directory)
print(os.getcwd())

/panfs/ccds02/nobackup/people/mfrost2/projects/AKNS
/nfs4m/css/curated01/icesat-2/data/ATLAS/ATL13.005


In [11]:
#make empty dataframe
df = pd.DataFrame(columns = ['cycle', 'id', 'size', 'type', 'cloud_flag', 'bkgrd_flag', 
                             'shallow_flag', 'wind_flag', 'rgt', 'seg_lat', 'seg_lon',
                             'snow_ice_flag', 'wave_flag', 'depth', 'anomalies', 'beam',
                             'start_date', 'qual0', 'qual1', 'qual2', 'qual3', 'sc_orient',
                             'beg_lat', 'beg_lon', 'end_lat', 'end_lon'])
print(df)

Empty DataFrame
Columns: [cycle, id, size, type, cloud_flag, bkgrd_flag, shallow_flag, wind_flag, rgt, seg_lat, seg_lon, snow_ice_flag, wave_flag, depth, anomalies, beam, start_date, qual0, qual1, qual2, qual3, orientation, beg_lat, beg_lon, end_lat, end_lon]
Index: []

[0 rows x 26 columns]


In [12]:
year = slice(6,10)
month = slice(10,12)
day = slice(12,14)


In [13]:
##Doing above, but as a function
# def subfolder(file):
#     year = slice(6,10)
#     month = slice(10,12)
#     day = slice(12,14)
#     subdir = str(hdfile[year]+"."+hdfile[month]+"."+hdfile[day]+"/"+hdfile)
#     print(subdir)

In [14]:
for hdfile in atl13_granule_list_nopath:
    
    subdir = str(hdfile[year]+"."+hdfile[month]+"."+hdfile[day]+"/"+hdfile)
        
    FILE_NAME = subdir
    print(FILE_NAME)
    #'C:\ICESat\granules3\processed_ATL13_20190709221119_01810401_005_01.h5'
    
    #Get the beams of the granule
    hf = h5py.File(FILE_NAME, 'r')
    keys = (list(hf.keys()))
    beams = [x for x in keys if x.startswith('gt')]
    
    #check beams
    # print(beams)
    
    #cycle through desired variables
    for beam in beams:
        cycle = str("/"+ beam + "/cycle")                       #Tracks the number of 91-day cycles in the mission, beginning with 01.
        water_id = str("/"+ beam + "/inland_water_body_id")     #HydroLAKES water body ID number
        water_size = str("/"+ beam + "/inland_water_body_size") #A=area, where 0=Not_Assigned, 1=A>10,000 sq km, 2=10,000>A>=1,000, 3=1,000>A>=100, 4=100>A>=10, 5=10>A>=1,6=1>A>=0.1, 7=0.01>A,
        water_type = str("/"+ beam + "/inland_water_body_type") #Type of Inland Water Body, where 1=Lake, 2=Known, Reservoir, 3=(Reserved for future use), 4=Ephemeral, Water, 5=River, 6=Estuary or Bay, 7=Coastal Water,
        cloud_flag = str("/"+ beam + "/layer_flag_atl09")       #Consolidated cloud flag: 0=likely clear, 1=likely cloudy
        bkgrd_flag = str("/"+ beam + "/qf_bckgrd")              #Describes the degree of background photons present in each short segment. 0:<=0.001, 1:<=0.01,2:<=0.05,3:<=0.1,4:<=0.3,5:<=0.5,6:>0.5
        shallow_flag = str("/"+ beam + "/qf_bias_em")           #The Electromagnetic Bias flag is set based on threshold checks for the estimated electromagnetic height bias.
        wind_flag = str("/"+ beam + "/qf_bias_fit")             #The height bias fit flag is set based on the value of the goodness of fit bias estimated as the difference between the centroid elevations of the 
                                                                # observed surface waterhistogram and fitted integrated water surface model histogram.
        anomalies = str("/"+ beam + "/qf_subsurf_anomaly")      #1 = Subsurface anomaly due to bottom likely; 2 = Subsurface signal may indicate bottom or
                                                                # other anomaly; 3 = Possible subsurface anomaly
        rgt = str("/"+ beam + "/rgt")                           #the track on the earth at which a specified unit vector within the observatory is pointed. 
        seg_lat = str("/"+ beam + "/segment_lat")               #Latitude of reporting location for all short segment statistics
        seg_lon = str("/"+ beam + "/segment_lon")               #Longitude of reporting location for all short segment statistics
        snow_ice_flag = str("/"+ beam + "/snow_ice_atl09")      #NOAA snow/ice flag scaled by ATL09 (0=ice-free water, 1=snow-free land, 2=snow, 3=ice
        wave_flag = str("/"+ beam + "/stdev_water_surf")        #Standard deviation of water surface, calculated over long segments 
        depth = str("/"+ beam + "/water_depth")                 #Depth from the mean water surface to detected bottom.
        qual = str("/"+ beam + "/segment_quality")              # num of photons 1: nominal; 2: possible afterpulse; 3: possible impulse response; 4: possible TEP
        beg_lat = str("/"+ beam + "/sseg_start_lat")            #Latitude at which the short segment begins.
        beg_lon = str("/"+ beam + "/sseg_start_lon")            #Longitude at which the short segment begins.
        end_lat = str("/"+ beam + "/sseg_end_lat")              #Latitude at which the short segment ends.
        end_lon = str("/"+ beam + "/sseg_end_lon")              #Longitude at which the short segment ends.
    
        #print(beam)
        
        with h5py.File(FILE_NAME, mode='r') as f:

             cyclevar = f[cycle]
             cycles = cyclevar[:]
             
             idvar = f[water_id]
             water_ids = idvar[:]
            
             sizevar = f[water_size]
             water_sizes = sizevar[:]
            
             typevar = f[water_type]
             water_types = typevar[:]            
         
             cloudvar = f[cloud_flag]
             clouds = cloudvar[:]            
          
             bkgrdvar = f[bkgrd_flag]
             bkgrds = bkgrdvar[:]            
         
             shallowvar = f[shallow_flag]
             shallows = shallowvar[:]            
         
             windvar = f[wind_flag]
             winds = windvar[:]            
         
             anomvar = f[anomalies]
             anoms = anomvar[:]            
         
             rgtvar = f[rgt]
             rgts = rgtvar[:]            
         
             slatvar = f[seg_lat]
             slats = slatvar[:]            
         
             slonvar = f[seg_lon]
             slons = slonvar[:]            
         
             sn_ic_var = f[snow_ice_flag]
             sn_ics = sn_ic_var[:]   
        
             wavevar = f[wave_flag]
             waves = wavevar[:]   
         
             depthvar = f[depth]
             depths = depthvar[:]
             units = depthvar.attrs['units']
             long_name = depthvar.attrs['long_name']
                
             blatvar = f[beg_lat]
             blats = blatvar[:]            
         
             blonvar = f[beg_lon]
             blons = blonvar[:] 
                
             elatvar = f[end_lat]
             elats = blatvar[:]            
         
             elonvar = f[end_lon]
             elons = elonvar[:] 
            
             date_time_var = f[str("/ancillary_data/granule_start_utc")]
             date_time = date_time_var[:]
            
             orient_var = f[str("/orbit_info/sc_orient")]
             orients = orient_var[:]
               
            
             qualvar = f[qual]
             qual0 = qualvar[:,0]
             qual1 = qualvar[:,1]
             qual2 = qualvar[:,2]
             qual3 = qualvar[:,3]
        
        #check lengths
        #print(len(water_ids), len(orients), len(date_time), len(qual0))

        df2 = pd.DataFrame({'cycle' : cycles, 'id' : water_ids, 'size' : water_sizes, 
                         'type' : water_types, 'cloud_flag' : clouds, 'bkgrd_flag' : bkgrds, 
                         'shallow_flag' : shallows, 'wind_flag' : winds, 'anomalies': anoms, 
                         'rgt' : rgts, 'seg_lat' : slats, 'seg_lon': slons, 'snow_ice_flag' : sn_ics, 
                         'wave_flag' : waves, 'depth' : depths, 'qual0' : qual0, 'qual1' : qual1, 
                         'qual2' : qual2, 'qual3' : qual3, 
                         'beg_lat' : blats, 'beg_lon': blons, 'end_lat': elats, 'end_lon' : elons})

        #include beam in dataset
        df2['beam'] = beam
        
        #Include datatetime in dataset
        df2['start_date'] = parser.parse(date_time[0], fuzzy=True)
        
        #Include orientation in dataset
        df2['sc_orient'] = orients[0]
        
        #bbox': [-168, 67.3, -141, 73]
        #filter for bounding box
        df2 = df2.loc[(df2['seg_lat'] >= 67.3) & (df2['seg_lat'] <= 73) & 
                      (df2['seg_lon'] <= -141) & (df2['seg_lon'] >= -168)]

        df = pd.concat([df,df2])
print(df.shape)
print(df.head)

2019.07.09/ATL13_20190709155410_01770401_005_01.h5
2019.07.09/ATL13_20190709221119_01810401_005_01.h5
2019.07.10/ATL13_20190710104537_01890401_005_01.h5
2019.07.10/ATL13_20190710231955_01970401_005_01.h5
2019.07.11/ATL13_20190711115414_02050401_005_01.h5
2019.07.12/ATL13_20190712002835_02130401_005_01.h5
2019.07.12/ATL13_20190712064544_02170401_005_01.h5
2019.07.12/ATL13_20190712130254_02210401_005_01.h5
2019.07.12/ATL13_20190712192004_02250401_005_01.h5
2019.07.13/ATL13_20190713013714_02290401_005_01.h5
2019.07.13/ATL13_20190713075423_02330401_005_01.h5
2019.07.13/ATL13_20190713202843_02410401_005_01.h5
2019.07.14/ATL13_20190714090302_02490401_005_01.h5
2019.07.14/ATL13_20190714213721_02570401_005_01.h5
2019.07.15/ATL13_20190715101141_02650401_005_01.h5
2019.07.15/ATL13_20190715224600_02730401_005_01.h5
2019.07.16/ATL13_20190716112019_02810401_005_01.h5
2019.07.16/ATL13_20190716235438_02890401_005_01.h5
2019.07.17/ATL13_20190717061148_02930401_005_01.h5
2019.07.17/ATL13_20190717122857

In [27]:
df['start_date'] = pd.to_datetime(df['start_date'])
df['start_date'].head()

24539   2019-07-09 22:11:19+00:00
24540   2019-07-09 22:11:19+00:00
24541   2019-07-09 22:11:19+00:00
24542   2019-07-09 22:11:19+00:00
24543   2019-07-09 22:11:19+00:00
Name: start_date, dtype: datetime64[ns, tzutc()]

In [24]:
df.shape

(1282216, 27)

In [17]:
df.to_csv(r'/explore/nobackup/people/mfrost2/projects/AKNS/data/ICESat_AKNS.csv', sep=',', index = False)

127927   2022-09-30 11:32:21+00:00
127928   2022-09-30 11:32:21+00:00
127929   2022-09-30 11:32:21+00:00
127930   2022-09-30 11:32:21+00:00
127931   2022-09-30 11:32:21+00:00
Name: start_date, dtype: datetime64[ns, tzutc()]